<a href="https://colab.research.google.com/github/ambreenraheem/5_days_AI_Agents_Training/blob/main/Gradio_AI_Career_Counselor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ambreen Abdul Raheem
(Power BI Data Analyst-Upwork Freelancer)\

MidTerm Project: AI Career Counselor Agent

Hugging Face Link: https://huggingface.co/spaces/ambreenraheem/AI_Career_Counselor

In [ ]:
import os
import requests
import gradio as gr
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# API headers
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": GEMINI_API_KEY
}

# Conversation memory
chat_history = []

# Career counselor system prompt (will be prepended to first user message)
SYSTEM_PROMPT = (
    "You are a professional, empathetic, and thorough global career counselor for students. "
    "When a student asks about careers or professions, respond with a structured, actionable plan. "
    "Include: 1) Overview, 2) Responsibilities, 3) Education/Pathways, 4) Skills, "
    "5) Salary & Outlook (as estimates), 6) Related roles, 7) Learning resources, "
    "8) Preparation by school stage, 9) Interview/CV tips, 10) 6–12 month personalized plan. "
    "Ask one clarifying question if the user's query is vague. "
    "Tone should be encouraging and clear. Tailor advice for the student's age & location if given."
)

def ensure_system_prompt():
    """Prepend system instructions to first user message (Gemini doesn't support 'system' role)."""
    if not chat_history:
        chat_history.append({
            "role": "user",
            "parts": [{"text": f"Instruction: {SYSTEM_PROMPT}"}]
        })

def build_user_message(user_input, student_context):
    """Builds the user's message with optional student context."""
    if student_context.strip():
        return {
            "role": "user",
            "parts": [{"text": f"Student context: {student_context.strip()}\nQuestion: {user_input.strip()}"}]
        }
    else:
        return {"role": "user", "parts": [{"text": user_input.strip()}]}

def generate_reply(user_input, student_context=""):
    """Send request to Gemini API and return the career advice."""
    ensure_system_prompt()
    chat_history.append(build_user_message(user_input, student_context))

    body = {
        "contents": chat_history,
        "generationConfig": {
            "temperature": 0.3,
            "topK": 1,
            "topP": 0.95,
            "maxOutputTokens": 2048  # Allow longer answers
        }
    }

    try:
        response = requests.post(GEMINI_URL, headers=headers, json=body, timeout=30)
    except Exception as e:
        return f"**Error:** Request failed — {str(e)}"

    if response.status_code == 200:
        result = response.json()

        # Safe text extraction
        try:
            parts = result["candidates"][0]["content"].get("parts", [])
            if parts:
                reply = parts[0]["text"]
            else:
                reply = "_(No content returned — possibly cut off due to token limit)_"
        except Exception as e:
            reply = f"**Error parsing model reply:** {str(e)}"

        chat_history.append({"role": "model", "parts": [{"text": reply}]})
        return reply
    else:
        return f"**Error {response.status_code}:** {response.text}"

# Minimal clean interface using gr.Interface
demo = gr.Interface(
    fn=generate_reply,
    inputs=[
        gr.Textbox(label="Ask the Career Counselor", placeholder="E.g., I like biology, what careers fit me?"),
        gr.Textbox(label="Optional Student Context", placeholder="E.g., 15 years old, Pakistan, likes coding", lines=2)
    ],
    outputs=gr.Markdown(),
    title="Career Counselor Agent (Gemini 2.5 Flash)",
    description=(
        "Ask about careers, professions, study paths, and personalised plans for students. "
        "You can provide optional details like age, grade, interests, and country for tailored advice."
    )
)

if __name__ == "__main__":
    if not GEMINI_API_KEY:
        print("Warning: GEMINI_API_KEY not found in environment. Set it in your Hugging Face Space or .env file.")
    demo.launch(share=True)

requirements.txt

In [ ]:
huggingface_hub==0.25.2
python-dotenv
gradio
requests
websockets